In [ ]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Simple maps in `geopandas`
First I'll make a simple Wellington dataset. This is mostly refamiliarisation with things we've been covering.

In [ ]:
geographies = pd.read_csv("data/geographic-areas-table-2023.csv")[
    ["SA22023_code", "SA22023_name", "SA12023_code"]]

In [ ]:
welly = gpd.read_file("data/sa1-wellington.gpkg")
welly.SA12023_V1_00 = welly.SA12023_V1_00.astype("int64")

In [ ]:
welly = welly.merge(geographies, 
                    left_on = "SA12023_V1_00", right_on = "SA12023_code") \
   .rename(mapper = {"SA22023_code": "sa2_code", 
                     "SA22023_name": "name"}, axis = "columns") \
   .drop(columns = ["SA12023_code"])

welly

One new thing here: the `dissolve` operation allows us to combine (in this case) SA1s into SA2s, and the `aggfunc` parameter can specify what functions to use on each variable (using a dictionary), when calculating values for the dissolved areas from data in the source areas. Here we want to sum populations, but just pick the first value for the SA2 names. 

In [ ]:
pop_data = pd.read_csv("data/sa1-pops.csv")
pop_data

In [ ]:
welly = welly.merge(pop_data) \
    .dissolve(by = "sa2_code", aggfunc = {"CURPop": "sum", 
                                          "name": "first"})
welly["pop_density"] = welly.CURPop / welly.geometry.area * 1000000

welly

We'll also save this for later use.

In [ ]:
welly.to_file("data/sa2-wellington.gpkg", overwrite = True)

## Static maps using `matplotlib`
`geopandas` default mapping tool is `matplotlib` which is frankly... pretty horrible! Since you work in a GIS team it's unlikely you'll be using it to make final maps for publication, but you can make very useful exploratory maps for analysis and also for inclusion in regular reports.

In the cells below, I show how we can make a simple map, and then progressively add various refinements to make it (potentially!) useful. The basic plotting function isn't great, but provides a useful sanity check (did I load the right data? Are there missing values?)

In [ ]:
welly.plot()
plt.show()

Would be nice it it was bigger... control that with the `figsize` parameter, which is width, height in inches as a tuple. The dimensions aren't really 'real' unless you export things out to a file, but you can still use this parameter to make the map a better size for your purposes.

In [ ]:
welly.plot(figsize = (12, 12))
plt.show()

We can specify fill colour, edge colour, and line width. In the code I use abbreviated parameter names. The full names are `facecolor`, `edgecolor`, and `linewidth`. Colours can be specified using any of `matplotlib`'s many options as [detailed here](https://matplotlib.org/stable/users/explain/colors/colors.html#colors-def).

In [ ]:
welly.plot(figsize = (12, 12), fc = "darkgrey", ec = "white", lw = 0.5)
plt.show()

Labelling this with the names of SA2s is possible, but a bit roundabout. We can't label a whole bunch of points at once, but instead have to iterate over them and them one at a time using the `text()` method of the `Axes` object on which our map has been plotted. We get access to the `Axes` object by assigning our plot to a variable, which gives us a 'handle' on the plot, which we use to add additional layers, or otherwise change things. Below, I've used its `set_axis_off()` method to remove the x and y axes.

In [ ]:
ax = welly.plot(figsize = (12, 12), fc = "lightgrey", ec = "white", lw = 0.5)

for pt, label in zip(welly.geometry.representative_point(), welly.name):
    ax.text(x = pt.x, y = pt.y, s = label, 
            fontsize = 6, ha = "center", va = "center")
ax.set_axis_off()

plt.show()

If you really want to label maps in python, I recommend developing lists of named locations as separate data layers!

## Thematic maps
Probably, you have data which you'd like to map. This is something that `geopandas` does a reasonable job of, via `matplotlib` and also assuming that you have `mapclassify` installed in the environment. You have to specify the variable on which any variation in colours is to be based, along with a 'colour map' name to be used. By default, this will give an unclassed choropleth, where colours are assigned on a continuous scale. Available colour maps are [listed here](https://matplotlib.org/stable/users/explain/colors/colormaps.html). When you don't specify a classification `scheme` (see below) I'm afraid you are stuck with the colour ramp format of legend.

In [ ]:
ax = welly.plot(column = "pop_density", cmap = "Purples", ec = "k", lw = 0.5, 
                figsize = (12, 12), legend = True)
ax.set_axis_off()

plt.show()

`mapclassify` makes many options available for classification, which we very skew data like these can be useful. By the way, you can see just how skew by choosing a different kind of plot:

In [ ]:
welly.plot(kind = "hist", column = "pop_density", 
           color = "lightgrey", edgecolor = "k")

plt.show()

Anyway, back to maps... assuming `mapclassify` is installed, specify `scheme` and `k` (the number of classes) to get the classification you want.

In [ ]:
ax = welly.plot(column = "pop_density", cmap = "Purples", 
                scheme = "quantiles", k = 7,
                ec = "k", lw = 0.5, figsize = (12, 12), 
                legend = True, 
                legend_kwds = {"title": "Pop density per sq km",
                               "loc": "upper left"})
ax.set_axis_off()
plt.show()

The available classification schemes are [documented here](https://pysal.org/mapclassify/api.html).

## Base maps
You can get static web basemaps using [`contextily`](https://contextily.readthedocs.io/en/latest/index.html). Here's a simple example.

In [ ]:
import contextily as cx

ax = welly.plot(fc = "None", ec = "k", lw = 0.5, figsize = (8, 8))
ax.set_axis_off()

cx.add_basemap(ax, crs = welly.crs, source=cx.providers.CartoDB.Voyager)

## Web maps
There are many situations where small(ish) static maps are not so useful because you need to inspect particular parts of the map up close. Web maps are one way to resolve this problem. I'm not going to get into this in any detail, but the `geopandas` `GeoDataFrame.explore()` method provides a simple way to quickly make a web map. It's important to realise that this may not scale well to datasets with many thousands of shapes. Alternative modules that better handle large datasets are [`lonboard`](https://github.com/developmentseed/lonboard) and [`pydeck`](https://deckgl.readthedocs.io/en/latest/) but these are really out of scope for these sessions. 

Of course... there is always the option to inspect data in a GIS.

In [ ]:
welly.explore(tiles = "CartoDB.Positron", column = "CURPop", cmap = "Reds",
              scheme = "equalinterval", k = 5,
              tooltip = "name", popup = ["CURPop", "pop_density"], 
              style_kwds = {"weight": 0.5, "color": "white"})